In [ ]:
# 1. find the any substring in a genome context decoding a target amino acid sequence, including gene substring and its reverse complement
file_path = "./dataset_35902_6.txt"

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read the entire content of the file as a single string
    lines = file.readlines()

Geneseq = ""
for i in range(len(lines)-1):
    Geneseq += lines[i]
aa = lines[-1][:-1]

genetic_code = {
    'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
    'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
    'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M',
    'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
    'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
    'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'UAU': 'Y', 'UAC': 'Y', 'UAA': '', 'UAG': '',
    'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'UGU': 'C', 'UGC': 'C', 'UGA': '', 'UGG': 'W',
    'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
}

# write a function to generate all possible combinations of codons
def generate_all_combinations(dataDict, seq):
    possibleSubstring = []
    # base case: there is only the last aa in the seq
    if len(seq) == 1:
        return [code for code in dataDict[seq]]
    firstLetter = seq[0]
    remaining_seq = seq[1:]
    
    remaining_combination = generate_all_combinations(dataDict, remaining_seq)
    
    for codon in dataDict[firstLetter]:
        for combinations in remaining_combination:
            substring = codon + combinations
            possibleSubstring.append(substring)

    return possibleSubstring


# a function that transform a given string to its reverse_complement
def reverse_complement(stringList):
    result = []
    baseDict = {"A":"T","T":"A","C":"G","G":"C"}
    for s in stringList:
        reverse = s[::-1]
        complement = ""
        for letter in reverse:
            letter = baseDict[letter]
            complement += letter
        result.append(complement)
    return result

# generate a dictionary store the corresponding codon for each amino acid in the aa seq.
codeDict = {}
for i in range(len(aa)):
    if aa[i] not in codeDict.keys():
        # transform from U to T
        code = [code.replace("U","T") for code in genetic_code.keys() if genetic_code[code] == aa[i]]
        codeDict[aa[i]] = []
        codeDict[aa[i]].extend(code)

# this is the set of genes that decoding the target aa directly
possibleSubstring = generate_all_combinations(codeDict,aa)
reverseComplement = reverse_complement(possibleSubstring)
possibleSubstring.extend(reverseComplement)


substrings = []
stringLength = len(aa)*3
for i in range(len(Geneseq)-stringLength+1):
    if Geneseq[i:i+stringLength] in possibleSubstring:
        substring = Geneseq[i:i+stringLength]
        substrings.append(substring)

for i in substrings:
    print(i)

# 2. given a cyclic DNA of length n, how many substrings might it generate
# it have n start point for substrings of any length, it has n-1 possible lengths for substrings, so the number should be n*(n-1)


# 3. generate the cyclospectrum of a given peptide
amino_acid_weights = {
    'G': 57,
    'A': 71,
    'S': 87,
    'P': 97,
    'V': 99,
    'T': 101,
    'C': 103,
    'I': 113,
    'L': 113,  # Note: I and L have the same weight
    'N': 114,
    'D': 115,
    'K': 128,
    'Q': 128,  # Note: K and Q have the same weight
    'E': 129,
    'M': 131,
    'H': 137,
    'F': 147,
    'R': 156,
    'Y': 163,
    'W': 186  
}

def subPeptides(peptide):
    length = len(peptide)
    subpep = []
    # every start point
    for j in range(1, length):
        # every possible length of subpeptides
        for i in range(length): 
            if i+j <= length: 
                subpep.append(peptide[i:i+j])
            else:
                end = i+j-length
                sub = peptide[i:]+peptide[:end]
                subpep.append(sub)
    return subpep

# this is to generate spectrum, all mass of subpep and sort them
def massCal(subpeptides, amino_acid_weights):
    massList= [0]
    for s in subpeptides:
        mass = 0
        for letter in s:
            mass += amino_acid_weights[letter]
        massList.append(mass)
    return sorted(massList)

# file_path = "./dataset_35904_4.txt"

# # Open the file in read mode
# with open(file_path, 'r') as file:
#     # Read the entire content of the file as a single string
#     peptide = file.read()

# peptide = peptide.strip('\n')
# subpeptide = subPeptides(peptide)

# subMass = massCal(subpeptide, amino_acid_weights)
# mass = 0
# for letter in peptide:
#     mass += amino_acid_weights[letter]
# subMass.append(mass)
# result = ' '.join(map(str, subMass))
# print(result)


#4. for linear subpeptide, it should be the sum from 1 to length(1+2+...+length) +1
#5. branching and bounding algo for cyclopeptide sequencing
def CyclopeptideSeqencing(Spectrum, amino_acid_weights):
    candidatePeptide = {''}
    finalPeptide = []

    while candidatePeptide:
        
        # the expand should stop when the peptide has been the same size as the expected one
        candidatePeptide = Expand(candidatePeptide.copy(), amino_acid_weights)
        pep_to_remove = set()
        # if "" in candidatePeptide:
        #     candidatePeptide.remove("")
        for pep in candidatePeptide:
            if pep == '':
                pep_to_remove.add(pep)
                continue

            if Mass(pep, amino_acid_weights) == max(Spectrum):
                if Cyclospectrum(pep, amino_acid_weights) == Spectrum and pep not in finalPeptide:
                    finalPeptide.append(pep)
                pep_to_remove.add(pep)
            elif not Consistent(Linearspectrum(pep, amino_acid_weights), Spectrum):
                pep_to_remove.add(pep)
            
        candidatePeptide -= pep_to_remove
    
    return finalPeptide


def Mass(pep, amino_acid_weights):
    mass = 0
    for letter in pep:
        mass += amino_acid_weights[letter]
    return mass

def Cyclospectrum(pep, amino_acid_weights):
    sub = subPeptides(pep)
    submass = massCal(sub, amino_acid_weights)
    submass.append(Mass(pep, amino_acid_weights))
    # result = ' '.join(map(str, subMass))

    return submass 
    # return result

def Linearspectrum(pep, amino_acid_weights):
    
    length = len(pep)
    subpep = []
    # every start point
    for j in range(1, length):
        # every possible length of subpeptides
        for i in range(length): 
            if i+j <= length: 
                subpep.append(pep[i:i+j])
            
    submass = massCal(subpep, amino_acid_weights)
    submass.append(Mass(pep, amino_acid_weights))

    return submass 

# massList is a tuple
def numSpectrum(massList):
    length = len(massList)
    spectrumList = [0]
    for j in range(1, length+1):
        for i in range(length):
            if i+j <= length:
                subpep = massList[i:i+j]
                spectrumList.append(sum(subpep))
    return sorted(spectrumList)

def Expand(candidate, amino_acid_weights):
    newCandidate = set()
    for p in candidate:
        
        for aa in amino_acid_weights.keys():
            expanded = p + aa
            newCandidate.add(expanded)
            
    return newCandidate

def NumExpand(candidate, new_alphabet):
    newCandidate = set()
   
   # candidate is start from {tuple([])}

    for p in candidate:
        expanded = []
        for i in p:
            expanded.append(i)

        for c in new_alphabet:
            copy = expanded.copy()
            copy.append(c)
            

            newCandidate.add(tuple(copy))
    return newCandidate


def Consistent(testSpectrum, experiment):
    if  set(testSpectrum).issubset(set(experiment)) == False:
        return False
    else:
        countA = {}
        for m in testSpectrum:
            if m not in countA.keys():
                countA[m] = testSpectrum.count(m)
        countB = {}
        for n in experiment:
            if n not in countB.keys():
                countB[n] = experiment.count(n)
        for keys in countA.keys():
            if countA[keys] > countB[keys]:
                return False
    return True


# file_path = 'dataset_35906_6.txt'
# with open(file_path,"r") as file:
#     spectrum = file.read()
# spectrum = spectrum.strip('\n')
# spectrum = list(map(int, spectrum.split()))

# print(spectrum)
# # Join the list elements using a comma delimiter (',')
# # result = ','.join(numbers_list)
# # spectrum = [0, 113, 128, 186, 241, 299, 314, 427]
# subPep = CyclopeptideSeqencing(spectrum, amino_acid_weights)
# print(subPep)
# # transform to number
# massList = set()
# for element in subPep:
#     expression = ''
#     for c in element:
#         expression += str(amino_acid_weights[c])
#         expression += '-'
#     expression = expression[:-1]
#     massList.add(expression)

# for e in massList:
#     print(e)

#6. score(peptide, spectrum)
# file_path = 'dataset_35907_3.txt'
# with open(file_path,"r") as file:
#     lines = file.readlines()

# peptide = lines[0].strip('\n')
# spectrum = []
# for i in range(1,len(lines)):
#     for number in lines[i].split():
#         spectrum.append(int(number))

# theoretical = Cyclospectrum(peptide,amino_acid_weights)

# # print(spectrum)
# score = 0
# for n in spectrum.copy():
#     if n in theoretical:
#         score += 1
#         theoretical.remove(n)
#         spectrum.remove(n)
        
# print(score)

# linear score
def Score(theoretical, spectrum):
    # theoretical = Linearspectrum(peptide,amino_acid_weights)
    score = 0
    Refercopy = spectrum.copy()
    testcopy = theoretical.copy()
    for n in spectrum:
        if n in testcopy:
            score += 1
            testcopy.remove(n)
            Refercopy.remove(n)
            
    return score

# 7. LeaderboardCyclopeptideSequencing(spectrum, N)
def LeaderBoardSequencing(spectrum, n):
    if not spectrum:  # Check if spectrum is empty
        return None 
    leaderBoard = {''}
    leaderPep = ''
    pep_to_remove = set()
    while leaderBoard:
        leaderBoard = Expand(leaderBoard,amino_acid_weights)      
        for peptide in leaderBoard:
            if Mass(peptide, amino_acid_weights) == max(spectrum):
                theoretical = Linearspectrum(peptide,amino_acid_weights)
                if Score(theoretical, spectrum) > Score(Linearspectrum(leaderPep,amino_acid_weights), spectrum):
                    leaderPep = peptide
            elif Mass(peptide, amino_acid_weights) > max(spectrum):
                pep_to_remove.add(peptide)
        leaderBoard -= pep_to_remove
        if len(leaderBoard) > n:    
            leaderBoard = Trim(leaderBoard, spectrum, n)
    return leaderPep

def Trim(board, spectrum, n):
    boardList = []
    scoreList = []
    for pep in board:
        boardList.append(pep)
        theoretical = Linearspectrum(pep,amino_acid_weights)
        scoreList.append(Score(theoretical, spectrum))
        
    sorted_scores = sorted(scoreList, reverse=True)
    # the score at N-th place
    # find if there is any tie
    Nindex = [x for x in range(len(sorted_scores)) if scoreList[x]>= sorted_scores[n]]
    newBoard = []
    for x in Nindex:
        newBoard.append(boardList[x])
    newBoard = set(newBoard)
    return newBoard

# file_path = 'dataset_35907_8.txt'
# with open(file_path,"r") as file:
#     content = file.readlines()
# n = content[0].strip('\n')
# n = int(n)
# spectrum = []
# for i in range(1,len(content)):
#     for number in content[i].split():
#         spectrum.append(int(number))
# print(n)
# result = LeaderBoardSequencing(spectrum, n)
# # transform to number
# expression = ''
# for c in result:
#     expression += str(amino_acid_weights[c])
#     expression += '-'
    
# print(expression[:-1])

# 8. calculate the convolution of a given spectrum
def Convolution(spectrum):
    convolution = []
    for i in range(len(spectrum)):
        for j in range(i+1, len(spectrum)):
            differ = spectrum[j]-spectrum[i]
            if differ > 0:
                convolution.append(differ)
    return convolution


# file_path = "./dataset_35909_4.txt"
# with open(file_path, "r") as file:
#     spectrum = file.read()
# spectrum = spectrum.strip('\n')
# spectrum = list(map(int, spectrum.split()))
# result = Convolution(spectrum)
# output_string = ' '.join(map(str, result))

# print(output_string)

# 9. ConvolutionCyclopeptideSequencing
def ConvolutionCyclopeptideSequencing(m, n, spectrum):
    # generate the new alphabet(only include the convolution value that is within 57-200)
    convolution = Convolution(spectrum)
    valid_convolution = [x for x in convolution if x <= 200 and x >= 57]
    count = {key: valid_convolution.count(key) for key in valid_convolution}
    cutVal = sorted(count.values(),reverse=True)[m]
    new_alphabet = [c for c in count.keys() if count[c] >= cutVal]
    
    
    # below is the leaderboard sequencing part, it is modified due to the new alphabet
    leaderBoard = {tuple([])}
    
    leaderPep = []
    pep_to_remove = set()
    while leaderBoard:
        # leaderBoard is now a set of tuples, rather than a set of strings
        leaderBoard = NumExpand(leaderBoard,new_alphabet) 
        for peptide in leaderBoard:
            if sum(peptide) == max(spectrum):
                theoretical = numSpectrum(peptide)
                if Score(theoretical, spectrum) > Score(numSpectrum(leaderPep), spectrum):
                    leaderPep = peptide
            elif sum(peptide) > max(spectrum):
                pep_to_remove.add(peptide)
        leaderBoard -= pep_to_remove
        if len(leaderBoard) > n:    
            leaderBoard = numTrim(leaderBoard, spectrum, n)
    theoretical1 = numSpectrum(list(leaderPep))
    print(Score(theoretical1,spectrum))
    print(theoretical1)
    print(leaderPep)
    theoretical2 = numSpectrum([99,71,137,57,72,57])
    print(Score(theoretical2,spectrum))
    print(theoretical2)
    return leaderPep

# an updated version of trim function considering board is now a set of tuples
def numTrim(board, spectrum, n):
    boardList = []
    scoreList = []
    for pep in board:
        boardList.append(pep)
        theoretical = numSpectrum(pep)
        scoreList.append(Score(theoretical, spectrum))
        
    sorted_scores = sorted(scoreList, reverse=True)
    # the score at N-th place
    # find if there is any tie
    Nindex = [x for x in range(len(sorted_scores)) if scoreList[x]>= sorted_scores[n]]
    newBoard = set()
    for x in Nindex:
        newBoard.add(boardList[x])
    newBoard = set(newBoard)
    return newBoard


file_path = './dataset_35909_7.txt'
with open(file_path,'r') as file:
    content = file.readlines()

m = content[0]
m = int(m)
n = content[1]
n = int(n)
spectrum = []
for i in range(2,len(content)):
    for number in content[i].split():
        spectrum.append(int(number))
result = ConvolutionCyclopeptideSequencing(m, n, spectrum)
output_string = '-'.join(map(str, result))
print(output_string)